In [14]:
from instance_runner import InstanceRunner
import pandas as pd
import json
from tqdm import tqdm
import os

In [15]:
runs = 10
output_file = "/home/pedro/cbm/output/results.csv"
instances = ["a1", "a2", "a3", "a4", "a5", "b1", "b2", "b3", "b4", "b5", "c1", "c2", "c3", "c4", "c5", "d1", "d2", "d3", "d4", "d5", "e1", "e2", "e3", "e4", "e5", "f1", "f2", "f3", "f4", "f5", "g1", "g2", "g3", "g4", "g5", "h1", "h2", "h3", "h4", "h5", "i1", "i2", "i3", "i4", "i5"]
soares = [215, 218, 208, 237, 231, 641, 637, 628, 648, 621, 1259, 1259, 1262, 1270, 1328, 495, 505, 446, 477, 459, 1445, 1434, 1420, 1417, 1465, 3006, 3052, 3051, 3061, 3078, 904, 896, 923, 945, 874, 2820, 2715, 2807, 2636, 2898, 5967, 6001, 5888, 5920, 5920]
haddadi = [210, 216, 204, 233, 226, 633, 631, 620, 638, 613, 1248, 1245, 1249, 1256, 1312, 491, 502, 445, 472, 454, 1420, 1406, 1399, 1391, 1440, 2959, 3010, 3006, 3017, 3032, 893, 884, 912, 932, 863, 2721, 2614, 2707, 2538, 2793, 5751, 5795, 5681, 5704, 5708]
parameters = {
    "threads": 4,
    "tempMin": 0.01,
    "tempMax": 2.25,
    "constructionBias": 3.0,
    "MKL": 400,
    "PTL": 2000,
    "tempD": 4,
    "upType": 1,
    "tempUpdate": 3,
    "movementType": 2
}

In [ ]:
file_exists = os.path.isfile(output_file)

for instance, s, h in tqdm(zip(instances, soares, haddadi)):
    for _ in range(runs):
        ir = InstanceRunner(
            parameters=parameters,
            root_path="/home/pedro/cbm",
            instance_path="/home/pedro/cbm/instances/" + instance,
            header_path="/home/pedro/cbm/src/CBMProblem.hpp",
        )
        output, elapsed = ir.run()
        output = json.loads(output)

        df_row = pd.DataFrame([{
            'instance': instance,
            'cost': output['cost'],
            'haddadi': h,
            'soares': s,
            'solution': output['solution'],
            't(s)': elapsed
        }])

        df_row.to_csv(output_file, mode='a', header=not file_exists, index=False)
        file_exists = True

0it [00:00, ?it/s]

8it [01:06,  8.94s/it]